In [30]:
# Imports
import pandas as pd
import numpy as np
from datasets import load_dataset
import nltk

In [31]:
dataset = load_dataset("stanfordnlp/imdb")

In [32]:
df_train = pd.DataFrame(dataset['train'])
df_train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
sentences = [text for text in df_train['text']]
countVectorizer = CountVectorizer()
X_test = countVectorizer.fit_transform(sentences)
X_test

<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3445861 stored elements in Compressed Sparse Row format>

In [34]:
Y_test = np.array(df_train['label'])
Y_test

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [40]:
# MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


clf = MLPClassifier()
print("Fitting...")
clf.fit(X_test, Y_test)
print("Fitted!")
y_test_pred = clf.predict(X_test)
print("Accuracy score:", accuracy_score(Y_test, y_test_pred))

Fitting...


C:\Users\jerem\UniversityClassProjects\CS178\venv\hw1-anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Fitted!


KeyboardInterrupt: 